In [1]:
import os
for dirname, _, filenames in os.walk('/root/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/root/pan20-celebrity-profiling-supplement-dataset-2020-02-28.zip
/root/celebrity_profiling_training-dataset_subset_2.csv
/root/Data Clearning_PreProcessing.ipynb
/root/.profile
/root/.bashrc
/root/.bash_history
/root/Team 12_ndjson_to_csv.ipynb
/root/glove.6B.50d.txt
/root/celebrity_profiling_training_after_preprocessing_5_tweets_2.csv
/root/prediction_BERT.ipynb
/root/glove.6B.200d.txt.filepart
/root/celebrity_profiling_training-dataset_subset_5_tweets.csv
/root/pan20-celebrity-profiling-training-dataset-2020-02-28.zip
/root/celebrity_profiling_training_after_preprocessing_5_tweets.csv
/root/Neural Network.ipynb
/root/tfidf_notebook.ipynb
/root/celebrity_profiling_training_after_preprocessing.csv
/root/pan20-celebrity-profiling-supplement-dataset-2020-02-28/celebrity-feeds.ndjson
/root/pan20-celebrity-profiling-supplement-dataset-2020-02-28/follower-feeds.ndjson
/root/pan20-celebrity-profiling-supplement-dataset-2020-02-28/labels.ndjson
/root/.pytorch_pretrained_bert/26bc1ad6c0ac742e

In [2]:
from sklearn.feature_selection import chi2

import argparse
import pandas as pd
import sys
import numpy as np
import pdb
from sklearn.metrics import make_scorer, f1_score, accuracy_score, recall_score, precision_score, classification_report, precision_recall_fscore_support
from sklearn.ensemble  import GradientBoostingClassifier, RandomForestClassifier
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import make_scorer, f1_score, accuracy_score, recall_score, precision_score, classification_report, precision_recall_fscore_support
from sklearn.utils import shuffle
from sklearn.ensemble  import GradientBoostingClassifier, RandomForestClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.utils import shuffle
import codecs
import operator
import gensim, sklearn
from collections import defaultdict
from nltk.tokenize import TweetTokenizer


/opt/conda/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [3]:
import pandas as pd
import numpy as np
from scipy.stats import randint
import seaborn as sns # used for plot interactive graph. 
import matplotlib.pyplot as plt
import seaborn as sns
from io import StringIO
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import chi2
from IPython.display import display
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.utils import resample

In [4]:
TOKENIZER = TweetTokenizer().tokenize

In [5]:
df=pd.read_csv("/root/celebrity_profiling_training_after_preprocessing_5_tweets_2.csv") 
#df=pd.read_csv("/root/celebrity_profiling_training_after_preprocessing.csv",usecols=['text','gender']) 



In [6]:
df.head()

,birthyear,gender,occupation,text
0,0,1,1,"i am in new zealand, an i must say there are a..."
1,0,0,0,rt so a one piece highschool anime is coming o...
2,0,0,0,rt kobe bryant the conductor comercial da nike...
3,3,1,1,rt this is an embarrassing night for america. ...
4,1,1,3,legoncitiesfc head coach with the accra sports...


In [7]:
def ML_Model_func(df,label):    
    min_label=min(df[label].value_counts().tolist())
    Label_set=list(set(df[label]))

    df_list=[]
    df_list_reduce=[]
    for i in Label_set:
        df_list.append(df[df[label]==i])


    for i in df_list:
        df_list_reduce.append(resample(i, 
                                     replace=False,    # sample without replacement
                                     n_samples=min_label,     # to match minority class
                                     random_state=123)) # reproducible results

    # Combine minority class with downsampled majority class
    df_total = pd.concat(df_list_reduce)

    # Display new class counts
    print(df_total[label].value_counts())
    df=df_total
    df=df.sample(frac=1).reset_index(drop=True)
    df=df.dropna()
    
    tweets=df.text.tolist()
    labels = df[label].tolist()
    Y=[]
    label0=[]
    for i in range(len(tweets)):
        Y.append(str(tweets[i]))
        label0.append(int(labels[i]))

    tfidf = TfidfVectorizer(sublinear_tf=True, min_df=3,max_features=10000, analyzer='word',ngram_range=(1, 2), stop_words='english')

    # We transform each complaint into a vector
    features = tfidf.fit_transform(Y)
    #np.asarray()


    print("Each of the %d complaints is represented by %d features (TF-IDF score of unigrams and bigrams)" %(features.shape))

    models = [
        RandomForestClassifier(n_estimators=100, max_depth=5, random_state=0),
        #GradientBoostingClassifier(n_estimators=100, max_depth=5, random_state=0),
        LinearSVC(),
        MultinomialNB(),
        LogisticRegression(multi_class='multinomial', solver="newton-cg"),
    ]

    # 5 Cross-validation
    CV = 5
    cv_df = pd.DataFrame(index=range(CV * len(models)))

    entries = []
    for model in models:
        model_name = model.__class__.__name__
        accuracies = cross_val_score(model, features, label0, scoring='f1_weighted', cv=CV)
        for fold_idx, accuracy in enumerate(accuracies):
            entries.append((model_name, fold_idx, accuracy))

    cv_df = pd.DataFrame(entries, columns=['model_name', 'fold_idx', 'f1_weighted'])


    mean_f1 = cv_df.groupby('model_name').f1_weighted.mean()
    std_f1 = cv_df.groupby('model_name').f1_weighted.std()

    acc = pd.concat([mean_f1, std_f1], axis= 1, 
              ignore_index=True)
    acc.columns = ['Mean f1', 'Standard deviation']
    print(acc)

In [8]:
df=pd.read_csv("/root/celebrity_profiling_training_after_preprocessing_5_tweets_2.csv") 


In [9]:
ML_Model_func(df,"gender")

1    191626
0    191626
Name: gender, dtype: int64
Each of the 383252 complaints is represented by 10000 features (TF-IDF score of unigrams and bigrams)
                         Mean f1  Standard deviation
model_name                                          
LinearSVC               0.562848            0.001589
LogisticRegression      0.563118            0.002032
MultinomialNB           0.562294            0.001347
RandomForestClassifier  0.519747            0.004653


In [10]:
df=pd.read_csv("/root/celebrity_profiling_training_after_preprocessing_5_tweets_2.csv") 


In [11]:
ML_Model_func(df,"occupation")

3    55797
2    55797
1    55797
0    55797
Name: occupation, dtype: int64
Each of the 223188 complaints is represented by 10000 features (TF-IDF score of unigrams and bigrams)
                         Mean f1  Standard deviation
model_name                                          
LinearSVC               0.405073            0.003243
LogisticRegression      0.407459            0.002616
MultinomialNB           0.400455            0.002055
RandomForestClassifier  0.309596            0.006565


In [12]:
df=pd.read_csv("/root/celebrity_profiling_training_after_preprocessing_5_tweets_2.csv") 


In [13]:
ML_Model_func(df,"birthyear")

4    43012
3    43012
2    43012
1    43012
0    43012
Name: birthyear, dtype: int64
Each of the 215060 complaints is represented by 10000 features (TF-IDF score of unigrams and bigrams)
                         Mean f1  Standard deviation
model_name                                          
LinearSVC               0.270562            0.001214
LogisticRegression      0.273701            0.000528
MultinomialNB           0.270436            0.001008
RandomForestClassifier  0.222645            0.008758


In [ ]:

df['occupation'].value_counts()

In [ ]:
df_majority = df[df.occupation==1]
df_majority2 = df[df.occupation==0]
df_minority = df[df.occupation==2]
df_majority3 = df[df.occupation==3]



In [ ]:
# Downsample majority class
df_majority_downsampled = resample(df_majority, 
                                 replace=False,    # sample without replacement
                                 n_samples=55797,     # to match minority class
                                 random_state=123) # reproducible results

In [ ]:
# Downsample majority class
df_majority_downsampled2 = resample(df_majority2, 
                                 replace=False,    # sample without replacement
                                 n_samples=55797,     # to match minority class
                                 random_state=123) # reproducible results

In [ ]:
# Downsample majority class
df_majority_downsampled3 = resample(df_majority3, 
                                 replace=False,    # sample without replacement
                                 n_samples=55797,     # to match minority class
                                 random_state=123) # reproducible results

In [ ]:
# Combine minority class with downsampled majority class
df_downsampled = pd.concat([df_majority_downsampled,df_majority_downsampled2,df_majority_downsampled3, df_minority])
 
# Display new class counts
df_downsampled.occupation.value_counts()

In [ ]:
df=df_downsampled

In [ ]:
df=df.sample(frac=1).reset_index(drop=True)

In [ ]:
df=df.dropna()

In [ ]:
tweets=df.text.tolist()

In [ ]:

#labels = df.occupation.tolist()
labels = df.occupation.tolist()


In [ ]:
labels[:10]

In [ ]:
text=[]
Y=[]
label0=[]
for i in range(len(tweets)):
    Y.append(str(tweets[i]))
    label0.append(int(labels[i]))


In [ ]:
tfidf = TfidfVectorizer(sublinear_tf=True, min_df=3,max_features=10000, analyzer='word',ngram_range=(1, 2), stop_words='english')

# We transform each complaint into a vector
features = tfidf.fit_transform(Y)
#np.asarray()


print("Each of the %d complaints is represented by %d features (TF-IDF score of unigrams and bigrams)" %(features.shape))

In [ ]:
type(features)

category_to_id = dict(df.values)
#id_to_category = dict(df[['text','gender']].values)
id_to_category = dict(df[['text','occupation']].values)

N = 3
for Product, category_id in sorted(category_to_id.items()):
    features_chi2 = chi2(features, labels == category_id)
    indices = np.argsort(features_chi2[0])
    feature_names = np.array(tfidf.get_feature_names())[indices]
    unigrams = [v for v in feature_names if len(v.split(' ')) == 1]
    bigrams = [v for v in feature_names if len(v.split(' ')) == 2]
    print("\n==> %s:" %(Product))
    print("  * Most Correlated Unigrams are: %s" %(', '.join(unigrams[-N:])))
    print("  * Most Correlated Bigrams are: %s" %(', '.join(bigrams[-N:])))

In [ ]:
models = [
    RandomForestClassifier(n_estimators=100, max_depth=5, random_state=0),
    #GradientBoostingClassifier(n_estimators=100, max_depth=5, random_state=0),
    LinearSVC(),
    MultinomialNB(),
    LogisticRegression(multi_class='multinomial', solver="newton-cg"),
]

# 5 Cross-validation
CV = 5
cv_df = pd.DataFrame(index=range(CV * len(models)))

entries = []
for model in models:
    model_name = model.__class__.__name__
    accuracies = cross_val_score(model, features, label0, scoring='f1_weighted', cv=CV)
    for fold_idx, accuracy in enumerate(accuracies):
        entries.append((model_name, fold_idx, accuracy))
    
cv_df = pd.DataFrame(entries, columns=['model_name', 'fold_idx', 'f1_weighted'])

In [ ]:
mean_f1 = cv_df.groupby('model_name').f1_weighted.mean()
std_f1 = cv_df.groupby('model_name').f1_weighted.std()

acc = pd.concat([mean_f1, std_f1], axis= 1, 
          ignore_index=True)
acc.columns = ['Mean f1', 'Standard deviation']
acc

In [ ]:
print("Next")

In [ ]:
X_train, X_test,Y_train,Y_test= train_test_split(features, label0, test_size=0.3, random_state=42)

In [ ]:
LogisticRegression_model = LogisticRegression(multi_class='multinomial', solver="newton-cg")
LogisticRegression_model.fit(X_train, Y_train)

In [ ]:
Y_pred = LogisticRegression_model.predict(features)
